# Part 4: Disaster Strikes

In [1]:
#q8
!hdfs dfsadmin -fs hdfs://boss:9000 -report

Configured Capacity: 25821052928 (24.05 GB)
Present Capacity: 19678228480 (18.33 GB)
DFS Remaining: 19403743232 (18.07 GB)
DFS Used: 274485248 (261.77 MB)
DFS Used%: 1.39%
Replicated Blocks:
	Under replicated blocks: 167
	Blocks with corrupt replicas: 0
	Missing blocks: 75
	Missing blocks (with replication factor 1): 75
	Low redundancy blocks with highest priority to recover: 167
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (1):

Name: 172.18.0.4:9866 (p4-dn-2.p4_default)
Hostname: bb88c4bdf380
Decommission Status : Normal
Configured Capacity: 25821052928 (24.05 GB)
DFS Used: 274485248 (261.77 MB)
Non DFS Used: 6126047232 (5.71 GB)
DFS Remaining: 19403743232 (18.07 GB)
DFS Used%: 1.06%
DFS Remaining%: 75.15%
Configured Cac

In [2]:
import time
time.sleep(30)

In [3]:
import requests
r = requests.get(f"http://boss:9870/webhdfs/v1/single.csv?op=GETFILEBLOCKLOCATIONS")
print(r)
containerDict = {}
print
for i in range(len(r.json().get("BlockLocations").get("BlockLocation"))):
    if (r.status_code == 403):
        id = "lost"
    else:
        print(r.json().get("BlockLocations").get("BlockLocation")[i])
        id = r.json().get("BlockLocations").get("BlockLocation")[i].get("hosts")[0]
        print(id)
    blocks = r.json().get("BlockLocations").get("BlockLocation")[i].get("length") / 1048576
    count = containerDict.setdefault(id, 0)
    if (count == None):
            count = 0
    containerDict.update({id: (count+blocks)});
containerDict

<Response [200]>
{'topologyPaths': ['/default-rack/172.18.0.4:9866'], 'corrupt': False, 'cachedHosts': [], 'names': ['172.18.0.4:9866'], 'offset': 0, 'hosts': ['bb88c4bdf380'], 'length': 1048576, 'storageTypes': ['DISK']}
bb88c4bdf380
{'topologyPaths': ['/default-rack/172.18.0.4:9866'], 'corrupt': False, 'cachedHosts': [], 'names': ['172.18.0.4:9866'], 'offset': 1048576, 'hosts': ['bb88c4bdf380'], 'length': 1048576, 'storageTypes': ['DISK']}
bb88c4bdf380
{'topologyPaths': ['/default-rack/172.18.0.4:9866'], 'corrupt': False, 'cachedHosts': [], 'names': ['172.18.0.4:9866'], 'offset': 2097152, 'hosts': ['bb88c4bdf380'], 'length': 1048576, 'storageTypes': ['DISK']}
bb88c4bdf380
{'topologyPaths': ['/default-rack/172.18.0.4:9866'], 'corrupt': False, 'cachedHosts': [], 'names': ['172.18.0.4:9866'], 'offset': 3145728, 'hosts': ['bb88c4bdf380'], 'length': 1048576, 'storageTypes': ['DISK']}
bb88c4bdf380
{'topologyPaths': ['/default-rack/172.18.0.4:9866'], 'corrupt': False, 'cachedHosts': [], 'na

IndexError: list index out of range

In [ ]:
#q10
count = 0
with hdfs.open_input_file("/single.csv") as f:
    reader = io.TextIOWrapper(io.BufferedReader(f))
    for i, line in enumerate(reader):
        if re.search(r'Single Family', line):
            count += 1
count